# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
file='./Output_data/cities.csv'
weather_df=pd.read_csv(file)
weather_df.head()

,Date,Country,City,Lat,Lon,Max Tem,Cloudiness,Humidity,Wind Speed
0,1586628128,CK,Avarua,-21.21,-159.78,78.8,65,100,2.24
1,1586628171,ZA,Bredasdorp,-34.53,20.04,70.0,100,82,3.36
2,1586628265,NZ,Mataura,-46.19,168.86,54.0,100,74,3.00
3,1586628265,US,Saint Paul Harbor,57.79,-152.41,39.2,90,80,17.22
4,1586628068,MX,San Patricio,19.22,-104.70,86.0,40,74,11.41


### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_df[["Lat", "Lon"]]
weight=weather_df["Humidity"].astype(float)

# Add Heatmap layer to map
fig=gmaps.figure(center=(44.0,-6.0),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=np.max(weight),
                                 point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [6]:
# Narrow down the cities to fit weather conditions and Drop any rows will null values
hotel_df=weather_df[(weather_df["Max Tem"] <75)&(weather_df["Max Tem"] > 60)&(weather_df["Wind Speed"]< 10.0)&(weather_df["Cloudiness"]==0)].reset_index(drop=True) 
hotel_df[:""]

,Date,Country,City,Lat,Lon,Max Tem,Cloudiness,Humidity,Wind Speed
0,1586628306,NaN,Lüderitz,-26.65,15.16,73.40,0,49,9.17
1,1586628104,UY,Paso de los Toros,-32.82,-56.52,73.69,0,38,5.97
2,1586628173,FR,Cherbourg-en-Cotentin,49.64,-1.61,66.20,0,45,3.36
3,1586628341,EG,Maţāy,28.42,30.78,68.32,0,25,6.29
4,1586628352,EG,Rosetta,31.40,30.42,62.60,0,72,8.05


### Hotel Map

In [15]:
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"]=""
hotel_df

,Date,Country,City,Lat,Lon,Max Tem,Cloudiness,Humidity,Wind Speed,Hotel Name
0,1586624559,EG,Beni Suef,29.07,31.10,67.51,0,30,6.96,
1,1586624571,IT,Forio,40.73,13.86,66.00,0,77,4.70,
2,1586624574,DE,Hofheim,50.09,8.45,73.00,0,21,3.36,


In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# Hit the Google Places API for each city's coordinates


# Store the first Hotel result into the DataFrame





In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map
